## Setup up CUDA for PyTorch
#### *Must download CUDA drive from https://developer.nvidia.com/cuda-downloads and make a venv after in anaconda, not before. Then run the kernel below. 

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118
#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu118
!pip3 install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 5.6 MB/s eta 0:07:15
     ---------------------------------------- 0.0/2.4 GB 7.7 MB/s eta 0:05:19
     ---------------------------------------- 0.0/2.4 GB 8.6 MB/s eta 0:04:44
     ---------------------------------------- 0.0/2.4 GB 9.3 MB/s eta 0:04:24
     ---------------------------------------- 0.0/2.4 GB 9.4 MB/s eta 0:04:21
     ---------------------------------------- 0.0/2.4 GB 9.2 MB/s eta 0:04:26
     ---------------------------------------- 0.0/2.4 GB 9.4 MB/s eta 0:04:19
     ---------------------------------------- 0.0/2.4 GB 9.5 MB/s eta 0:04:18
     ---------------------------------------- 0.0/2.4 GB 9.5 MB/s eta 0:04:17
     ---------------------------------------- 0.0/2.4 GB 9.6 MB/s eta 0:04:14
     ---------------------------------------- 0.0/2.4 GB 9.5 MB/s eta 0:04:17
     -------

In [2]:
!nvidia-smi

Sun Jan 12 17:14:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   59C    P8             15W /   85W |     906MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Setup SAM and YOLO

In [ ]:
!git clone https://github.com/facebookresearch/sam2.git && %cd sam2
!pip install -e .

In [7]:
!pip install -q  supervision jupyter_bbox_widget roboflow dataclasses-json

In [8]:
!pip install ultralytics

  Using cached ultralytics-8.3.59-py3-none-any.whl.metadata (35 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.13-py3-none-any.whl.metadata (9.4 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached ultralytics-8.3.59-py3-none-any.whl (906 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached ultralytics_thop-2.0.13-py3-none-any.whl (26 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


## Other Pip installs

In [9]:
!pip install imageio

  Using cached imageio-2.36.1-py3-none-any.whl.metadata (5.2 kB)
Using cached imageio-2.36.1-py3-none-any.whl (315 kB)


## Setup Computer and Dependencies

In [1]:
import torch
print(torch.__version__)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

2.5.1+cu121
cuda:0


In [12]:
%cd ~
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

young_checkpoint = r"C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\V2 Models\best_V2_Mature_SAM2.pth"
mature_checkpoint = r"C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\V2 Models\best_V2_Mature_SAM2.pth"

model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

young_predictor = SAM2ImagePredictor(build_sam2(model_cfg, young_checkpoint))
mature_predictor = SAM2ImagePredictor(build_sam2(model_cfg, mature_checkpoint))

C:\Users\richa_0


#### Add more dependencies

In [3]:
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
import matplotlib.pyplot as plt
import matplotlib.patches as patches 
import torchvision
import os
import ultralytics
from ultralytics import YOLO
from ultralytics import SAM
from matplotlib import pyplot as plt
import pandas as pd
import torch
import supervision as sv
from roboflow import Roboflow
import imageio.v3 as iio
import datetime
import json
import cv2
import gc

## Define Methods

In [4]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.2]) #0.6
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375): #Extra method for the SAM inputs that we don't use
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax): #For debugging purposes to check if YOLO V8 is working
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    rect = plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)
    ax.add_patch(rect)

def show_polygon(polygons, ax):
    formatted_polygons = [np.array(polygon).reshape(-1, 2) for polygon in polygons]
    for polygon in formatted_polygons:
        polygon_patch = patches.Polygon(polygon, closed=True, edgecolor='red', facecolor=(0,0,0,0), lw=2)
        ax.add_patch(polygon_patch)

def show_ground_truth(area, ax):
    ax.text(50, 50, str(area) + " µm", color='white', fontsize=16, backgroundcolor='black')

def get_area(mask): #Prints the mask onto the image in a blue color
    true_count = np.count_nonzero(mask)
    return true_count

def mask_to_polygon(mask):
    # Convert mask to binary if it is not already
    if mask.max() > 1:
        _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    else:
        binary_mask = (mask * 255).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        # Simplify the contour to reduce the number of points
        epsilon = 0.001 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Extract points and flatten the list
        polygon = approx.reshape(-1, 2).tolist()
        flat_polygon = [point for sublist in polygon for point in sublist]
        polygons.append(flat_polygon)

    return polygons

def get_physical_tag(image): #Gets the physical conversion ratios through a library
    physical_size_x = 1
    physical_size_y = 1
    exifdata = image.getexif()
    for tagid in exifdata:
        tagname = TAGS.get(tagid, tagid)
        if tagname == "ImageDescription":
            try:
                value = exifdata.get(tagid)
                start_index_x = value.find('PhysicalSizeX="') + len('PhysicalSizeX="')
                end_index_x = value.find('"', start_index_x)
                physical_size_x = value[start_index_x:end_index_x]
                start_index_x_unit = value.find('PhysicalSizeXUnit="') + len('PhysicalSizeXUnit="')
                end_index_x_unit = value.find('"', start_index_x_unit)
                physical_size_x_unit = value[start_index_x_unit:end_index_x_unit]
                start_index_y = value.find('PhysicalSizeY="') + len('PhysicalSizeY="')
                end_index_y = value.find('"', start_index_y)
                physical_size_y = value[start_index_y:end_index_y]
                start_index_y_unit = value.find('PhysicalSizeYUnit="') + len('PhysicalSizeYUnit="')
                end_index_y_unit = value.find('"', start_index_y_unit)
                physical_size_y_unit = value[start_index_y_unit:end_index_y_unit]
            except Exception as e:
                print(f'Failed to process {image}: {e}')
                continue

    if (physical_size_x == 1):
        print("Couldn't get Metadata with method 2")

    return float(physical_size_x), float(physical_size_y)

def get_physical(image_path):
    physical_size_x = 1
    physical_size_y = 1
    try:
        # Retrieve metadata
        metadata = iio.immeta(image_path)

        # Debugging output to understand metadata structure
        #print(f"Metadata for {image_path}: {metadata}")

        # Ensure the 'pixelsizex' and 'pixelsizey' keys are present in the metadata
        physical_size_x = metadata.get('pixelsizex', 'N/A')
        physical_size_y = metadata.get('pixelsizey', 'N/A')

        # Check if the physical sizes are retrieved correctly
        if physical_size_x == 'N/A' or physical_size_y == 'N/A':
            physical_size_x = 1
            physical_size_y = 1
        else :
            physical_size_x = float(physical_size_x) * 10**6
            physical_size_y = float(physical_size_y) * 10**6
            
    except Exception as e:
        print(f'Failed to retrieve metadata from {image_path}: {e}')

    if (physical_size_x == 1):
        print("Couldn't get Metadata with method 1")

    return physical_size_x, physical_size_y

def create_coco_json(images_data, annotations, categories, output_file): #Creates a COCO.json file that allows user to edit annotations else where
    coco_json = {
        "info": {
            "description": "Coral Areas",
            "url": "https://drive.google.com/drive/folders/1I2PxS79XVj3VTLM-lKzeGh2WqyUNf9IB?usp=sharing",
            "version": "1.0",
            "year": 2024,
            "contributor": "Richard Zhao, Eric Su, Simon Zhao, Tracy Chen",
            "date_created": "2024-07-25"
        },
        "licenses": [
            {
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License",
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
            }
        ],
        "images": [],
        "annotations": [],
        "categories": categories
    }

    for image in images_data:
        image_info = {
            "id": image["id"],
            "license": image["license"],
            "file_name": image["file_name"],
            "width": image["width"],
            "height": image["height"],
            "date_captured": image["date_captured"]
        }
        coco_json["images"].append(image_info)

    for annotation in annotations:
        annotation_info = {
            "id": annotation["id"],
            "image_id": annotation["image_id"],
            "category_id": annotation["category_id"],
            "bbox": annotation["bbox"],
            "area": annotation["area"],
            "segmentation": annotation["segmentation"],
            "iscrowd": annotation["iscrowd"]
        }
        coco_json["annotations"].append(annotation_info)

    with open(output_file, 'w') as f:
        json.dump(coco_json, f, indent=4)

In [5]:
def execute_block1(image_path): #Opens the image and returns the bounding box
    image = Image.open(image_path)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    model = YOLO("C:/Users/richa_0/Documents/Coral Research/Machine Learning Models/V2 Models/best_2733_augmented_adjusted_YOLOV11.pt")
    results = model.predict(image, verbose=False)
    classes = ['late-recruits']
    for result in results:
        boxes = result.boxes
    if boxes.xyxy.tolist():
        bbox = boxes.xyxy.tolist()[0]
        class_ids = boxes.cls.tolist()
        classes = [model.names[int(cls_id)] for cls_id in class_ids]
    else:
        bbox = [0, width, 0, height]
    return image, bbox, classes

def execute_block2(image, bbox, classes): #Gets the mask using the image and the bounding box
    input_box = np.array(bbox)
    center = [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]
    top_mid = [(bbox[0] + bbox[2]) / 2, bbox[1]]
    bottom_mid =[(bbox[0] + bbox[2]) / 2, bbox[3]]
    left_mid = [bbox[0], (bbox[1] + bbox[3]) / 2]
    right_mid = [bbox[2], (bbox[1] + bbox[3]) / 2]
    if len(classes) > 0 and classes[0] == 'rah': #switch for ablation testing 'late-recruits'
        print("4 BG + Young")
        input_point = np.array([center, top_mid, bottom_mid, left_mid, right_mid])
        input_label = np.array([1, 0, 0, 0, 0])
        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            young_predictor.set_image(np.array(image))
            masks, _, _ = young_predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                box=input_box[None, :],
                multimask_output=False,
            )
    else:
        print("Mature")
        input_point = np.array([center])
        input_label = np.array([1])
        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            mature_predictor.set_image(np.array(image))
            masks, _, _ = mature_predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                box=input_box[None, :],
                multimask_output=False,
            )
    return masks

def execute_block3(image, bbox, masks, annotated_path, index, image_path): #Saves the images into the defined path, and gets the amount of pixels in the mask
    pixels = get_area(masks[0])
    pixel_size_x, pixel_size_y = get_physical(image_path)
    if (pixel_size_x == 1):
        pixel_size_x, pixel_size_y = get_physical_tag(image)
    area = pixel_size_x * pixel_size_y * pixels
    segmentation = mask_to_polygon(masks[0])
    image_file = os.path.basename(annotated_path)
    input_box = np.array(bbox)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    scale_factor = 100
    fig, ax = plt.subplots(figsize=(width / scale_factor, height / scale_factor))
    ax.set_position([0, 0, 1, 1])
    ax.set_axis_off()
    ax.imshow(image, aspect='auto')
    #show_mask(masks[0], ax)
    show_box(input_box, ax)
    show_polygon(segmentation, ax)
    show_ground_truth(area, ax)
    fig.savefig(annotated_path, dpi=100)
    plt.close(fig)

    image_info = {
        "id": index,
        "license": 1,
        "file_name": image_file,
        "height": height,
        "width": width,
        "date_captured": datetime.datetime.now().isoformat()
    }
    annotation_info = {
        "id": index,
        "image_id": index,
        "category_id": 1,
        "bbox": bbox,
        "area": pixels,
        "segmentation": segmentation,
        "iscrowd": 0
    }

    return pixels, image_info, annotation_info, area, pixel_size_x, pixel_size_y

## Define Important Variables

In [14]:
folder_read_paths = [
    "C:/Users/richa_0/Documents/Coral Research/Test 4/"
]

folder_write_path = "C:/Users/richa_0/Documents/Coral Research/Code Output/"
csv_path = folder_write_path + "coral_areas_output.csv"
columns = ["Folder", "Image Name", "Class", "Pixel Area", "Pixel Size X", "Pixel Size Y", "µm^2"]

In [ ]:
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_Mote_TIF\Year 1\W15\CNAT-B",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_Mote_TIF\Year 1\W15\CNAT-R",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_Mote_TIF\Year 1\W15\PSTR-B",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_Mote_TIF\Year 1\W15\PSTR-R",

    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 1\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 2\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 3\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 4\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 5\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 6\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 7\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 8\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 9\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 10\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 11\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 12\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 13\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 14\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 15\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 16\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 17\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 18\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 19\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 20\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 21\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 22\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 23\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 24\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 25\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 26\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 27\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 28\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 29\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 30\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 31\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 32\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 33\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 34\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 35\measurements",
    r"C:\Users\richa_0\Documents\Coral Research\Images\Input Images\Original_Raw_BluelightCellSens_TIF\Blue Light CellSens\Week 4 10_25_23\tank 36\measurements",

## Run Area Generator

In [15]:
unique_folder_names = {}

for index, folder in enumerate(folder_read_paths):
    images_data = []
    annotations = []
    categories = [
        {"id":0,"name":"coral","supercategory":"none"},
        {"id":1,"name":"coral","supercategory":"coral"}
    ]
    folder_name = os.path.basename(folder.rstrip('/'))

    if folder_name in unique_folder_names:
        unique_folder_names[folder_name] += 1
        new_folder_name = f"{folder_name}_{unique_folder_names[folder_name]}"
    else:
        unique_folder_names[folder_name] = 1
        new_folder_name = folder_name + "_1"

    new_folder_path = os.path.join(folder_write_path, new_folder_name)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    json_path = os.path.join(new_folder_path, "_annotations.coco.json")

    image_index = 0
    for image_file in os.listdir(folder):
        if image_file.endswith('.tif'):
            image_path = os.path.join(folder, image_file)
            base_name, _ = os.path.splitext(image_file)
            image_file_png = base_name + ".png"
            annotated_path = os.path.join(new_folder_path, image_file_png)
            image, bbox, classes = execute_block1(image_path)
            masks = execute_block2(image, bbox, classes)
            pixels, image_info, annotation_info, area, pixel_size_x, pixel_size_y = execute_block3(image, bbox, masks, annotated_path, image_index, image_path)
            images_data.append(image_info)
            annotations.append(annotation_info)
            new_row = {
                "Folder": new_folder_name,
                "Image Name": image_file,
                "Class": classes[0] if classes else "No Class Detected",  # Default to a string if classes is empty
                "Pixel Area": pixels,
                "Pixel Size X": pixel_size_x,
                "Pixel Size Y": pixel_size_y,
                "µm^2": area
            }
            new_row_df = pd.DataFrame([new_row], columns=columns)
            new_row_df.to_csv(csv_path, mode='a', header=not os.path.exists(csv_path), index=False)
            print(
                "Folder: " + new_folder_name + "\n" +
                "Image Name: " + image_file + "\n" +
                "Class: " + (str(classes[0]) if classes else "No Class Detected") + "\n" +
                "Bounding Box: " + str(bbox) + "\n" +
                "Pixels: " + str(pixels) + "\n" +
                "Pixel Size X: " + str(pixel_size_x) + "\n" +
                "Pixel Size Y: " + str(pixel_size_y) + "\n" +
                "Area: " + str(area) + " µm^2" + "\n"
            )
            image_index += 1
            gc.collect()
            torch.cuda.empty_cache()

    create_coco_json(images_data, annotations, categories, json_path)

Mature
Couldn't get Metadata with method 1
Couldn't get Metadata with method 2
Folder: Test 4_1
Image Name: 101_1_20231219.tif
Class: late-recruits
Bounding Box: [455.59063720703125, 99.36758422851562, 1375.2513427734375, 976.0171508789062]
Pixels: 634353
Pixel Size X: 1.0
Pixel Size Y: 1.0
Area: 634353.0 µm^2

Mature
Couldn't get Metadata with method 1
Couldn't get Metadata with method 2
Folder: Test 4_1
Image Name: 101_1_20240130.tif
Class: late-recruits
Bounding Box: [429.65020751953125, 102.63986206054688, 1537.1466064453125, 976.5269775390625]
Pixels: 767618
Pixel Size X: 1.0
Pixel Size Y: 1.0
Area: 767618.0 µm^2

Mature
Couldn't get Metadata with method 1
Couldn't get Metadata with method 2
Folder: Test 4_1
Image Name: 101_1_20240305.tif
Class: late-recruits
Bounding Box: [564.9849853515625, 60.872039794921875, 1525.7236328125, 986.380126953125]
Pixels: 687366
Pixel Size X: 1.0
Pixel Size Y: 1.0
Area: 687366.0 µm^2

Mature
Couldn't get Metadata with method 1
Folder: Test 4_1
Imag